In [1]:
import os
import fnmatch
import numpy as np
import tensorflow as tf  # Updated import
import time
import scipy.io as scio
import matplotlib.pyplot as plt

2024-03-26 01:10:12.395100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_size = 150

In [3]:
def load_sca_model(model_folder, model_file):
    for f_name in os.listdir(model_folder):
        if f_name.startswith(model_file):
            model = tf.keras.models.load_model(model_folder+f_name)  # Updated for tf.keras
            print(f_name)
            print(model.summary())
        else:
            continue
    return model

In [4]:
def rank_func(model, dataset, labels):
    input_layer_shape = model.input_shape  # Updated for direct access
    if len(input_layer_shape) == 2:
        input_data = dataset
    elif len(input_layer_shape) == 3:
        input_data = dataset.reshape((dataset.shape[0], dataset.shape[1], 1))
    else:
        print("Error: model input shape length %d is not expected ..." % len(input_layer_shape))
        sys.exit(-1)

    predictions = model.predict(input_data)
    predictions = np.argmax(predictions, axis=1)
    ranks_prd = np.bitwise_xor(predictions, labels)
    return ranks_prd

In [5]:
def load_traces(start=7, end=10):
    # Import our traces
    # Choose here which files to use
    traces = np.load(data_folder + 'trace' + str(start) + '.npy')
    labels = np.load(data_folder + 'label' + str(start) + '.npy')
    for i in range(start+1, end):
        tempTrace = np.load(data_folder + 'trace' + str(i) + '.npy')
        tempLabel = np.load(data_folder + 'label' + str(i) + '.npy')
        traces = np.append(traces, tempTrace, axis=0)
        labels = np.append(labels, tempLabel, axis=0)

    print('traces shape:', traces.shape)
    print('labels shape:', labels.shape)

    # Scale (standardize) traces
    delimitedTraces = np.zeros(traces.shape)
    for x_index in range(traces.shape[0]):
        delimitedTraces[x_index, :] = -1 + (traces[x_index, :] - np.min(traces[x_index, :])) * 2 / (
                    np.max(traces[x_index, :]) - np.min(traces[x_index, :]))

    return delimitedTraces, labels

In [6]:
def check_model(model_folder, model, count, start=7, end=10):
    traces, labels = load_traces(start, end)
    model_load = load_sca_model(model_folder, model)
    ranks_prd = rank_func(model_load, traces, labels)
    a = 1-np.mean(ranks_prd)
    print('test accuracy:', a)
    print('mean of labels:', np.mean(labels))

In [7]:
data_folder = "../traces/puf12_avg100/testing/"
model_folder = "../models/puf12/"
modelName = 'puf12'

start = time.time()
print('input size =', input_size)
        					
count = 0
for file_name in os.listdir(model_folder):
    if fnmatch.fnmatch(file_name, '*.keras'):  # TensorFlow 2.x supports .h5 but consider SavedModel format
        model = file_name
        count += 1
        check_model(model_folder, model, count, 3, 5)

end = time.time()
print("The total running time was: ", ((end-start)/60), " minutes.")

input size = 150
traces shape: (20000, 150)
labels shape: (20000,)
puf12.keras


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 150)            │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           604 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4)              │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,077 (12.02 KB)

 Trainable params: 922 (3.60 KB)

...

 Total params: 3,111 (12.16 KB)

 Trainable params: 932 (3.64 KB)

 Non-trainable params: 312 (1.22 KB)

 Optimizer params: 1,867 (7.30 KB)

None
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step
test accuracy: 0.97585
mean of labels: 0.5916
The total running time was:  0.039864786465962726  minutes.


In [8]:
data_folder = "../traces/puf0_avg100/testing/"
model_folder = "../models/puf0/"
modelName = 'puf0'

start = time.time()
print('input size =', input_size)
        					
count = 0
for file_name in os.listdir(model_folder):
    if fnmatch.fnmatch(file_name, '*.keras'):  # TensorFlow 2.x supports .h5 but consider SavedModel format
        model = file_name
        count += 1
        check_model(model_folder, model, count)

end = time.time()
print("The total running time was: ", ((end-start)/60), " minutes.")

input size = 150
traces shape: (30000, 150)
labels shape: (30000,)
puf0.keras


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_2           │ (None, 150)            │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           604 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4)              │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_1 (Softmax)             │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,077 (12.02 KB)

 Trainable params: 922 (3.60 KB)

...

 Total params: 3,111 (12.16 KB)

 Trainable params: 932 (3.64 KB)

 Non-trainable params: 312 (1.22 KB)

 Optimizer params: 1,867 (7.30 KB)

None
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 579us/step
test accuracy: 0.7355666666666667
mean of labels: 0.5912333333333334
The total running time was:  0.05443445046742757  minutes.
